In [1]:
import pandas as pd
import numpy as np
import json
import urllib.request
import requests
import datetime
from datetime import datetime, timedelta,date
from bs4 import BeautifulSoup
import re
import dill
from ediblepickle import checkpoint
from retrying import retry
import os
from updates import get_response

In [2]:
tickers = dill.load(open('data/tickers.pkd', 'rb'))

### 1. Update news from Benzinga

In [4]:
cache_dir = 'data/BenzNewscache'
!rm data/BenzNewscache/*

In [5]:
@checkpoint(key=lambda args,kwargs:args[0], work_dir=cache_dir)
def update_news(ticker,date_last_update):
    url='https://www.benzinga.com/stock-articles/'+ticker+'/news'
    all_headlines={}
    next_page=''
    done=False
    while next_page!=None and not done:
        try:
            print(url)
            newspage=BeautifulSoup(get_response(url).text,'lxml')
        except NameError:
            return None
        news_list=newspage.find_all('div', attrs={'class':'item-list'})
        for item in news_list:
            if item.find('h3')!=None:
                date=datetime.strptime(item.find('h3').text,'%A, %B %d, %Y')
                if date>date_last_update:
                    headlines=item.find_all('span', attrs={'class':'field-content'})
                    for index,headline in enumerate(headlines):
                        headlines[index]=headline.find('a').text
                    all_headlines[date]=headlines
                else:
                    return all_headlines
        next_page=newspage.find('a', attrs={'title':"Go to next page"})
        if next_page==None:
            break
        else:
            url='https://www.benzinga.com'+next_page['href']
    return all_headlines

In [6]:
all_ticker_news = dill.load(open('data/all_ticker_news.pkd', 'rb'))
for ticker in tickers:
    if all_ticker_news[ticker]:
        date_last_update=max(all_ticker_news[ticker].keys())
        latest_news=update_news(ticker,date_last_update)
        all_ticker_news[ticker]=dict([x for x in latest_news.items()]+[x for x in all_ticker_news[ticker].items()])
    else:
        print('no news for ',ticker)
        

https://www.benzinga.com/stock-articles/SPY/news
https://www.benzinga.com/stock-articles/ABT/news
no news for  ABBV
https://www.benzinga.com/stock-articles/ABMD/news
https://www.benzinga.com/stock-articles/ACN/news
https://www.benzinga.com/stock-articles/ATVI/news
https://www.benzinga.com/stock-articles/ADBE/news
https://www.benzinga.com/stock-articles/AMD/news
https://www.benzinga.com/stock-articles/AAP/news
https://www.benzinga.com/stock-articles/AES/news
https://www.benzinga.com/stock-articles/AFL/news
https://www.benzinga.com/stock-articles/A/news
https://www.benzinga.com/stock-articles/APD/news
https://www.benzinga.com/stock-articles/AKAM/news
https://www.benzinga.com/stock-articles/ALK/news
https://www.benzinga.com/stock-articles/ALB/news
https://www.benzinga.com/stock-articles/ARE/news
https://www.benzinga.com/stock-articles/ALXN/news
https://www.benzinga.com/stock-articles/ALGN/news
https://www.benzinga.com/stock-articles/ALLE/news
https://www.benzinga.com/stock-articles/LNT/ne

https://www.benzinga.com/stock-articles/EQIX/news
https://www.benzinga.com/stock-articles/EQR/news
https://www.benzinga.com/stock-articles/ESS/news
https://www.benzinga.com/stock-articles/EL/news
https://www.benzinga.com/stock-articles/ETSY/news
https://www.benzinga.com/stock-articles/EVRG/news
https://www.benzinga.com/stock-articles/ES/news
https://www.benzinga.com/stock-articles/RE/news
https://www.benzinga.com/stock-articles/EXC/news
https://www.benzinga.com/stock-articles/EXPE/news
https://www.benzinga.com/stock-articles/EXPD/news
https://www.benzinga.com/stock-articles/EXR/news
https://www.benzinga.com/stock-articles/XOM/news
https://www.benzinga.com/stock-articles/FFIV/news
https://www.benzinga.com/stock-articles/FB/news
https://www.benzinga.com/stock-articles/FAST/news
https://www.benzinga.com/stock-articles/FRT/news
https://www.benzinga.com/stock-articles/FDX/news
https://www.benzinga.com/stock-articles/FIS/news
https://www.benzinga.com/stock-articles/FITB/news
https://www.benz

https://www.benzinga.com/stock-articles/NI/news
https://www.benzinga.com/stock-articles/NSC/news
https://www.benzinga.com/stock-articles/NTRS/news
https://www.benzinga.com/stock-articles/NOC/news
https://www.benzinga.com/stock-articles/NLOK/news
no news for  NCLH
https://www.benzinga.com/stock-articles/NRG/news
https://www.benzinga.com/stock-articles/NUE/news
https://www.benzinga.com/stock-articles/NVDA/news
https://www.benzinga.com/stock-articles/NVR/news
https://www.benzinga.com/stock-articles/ORLY/news
https://www.benzinga.com/stock-articles/OXY/news
https://www.benzinga.com/stock-articles/ODFL/news
https://www.benzinga.com/stock-articles/OMC/news
https://www.benzinga.com/stock-articles/OKE/news
https://www.benzinga.com/stock-articles/ORCL/news
https://www.benzinga.com/stock-articles/OTIS/news
https://www.benzinga.com/stock-articles/PCAR/news
https://www.benzinga.com/stock-articles/PKG/news
https://www.benzinga.com/stock-articles/PH/news
https://www.benzinga.com/stock-articles/PAYX/

In [7]:
all_ticker_news['MSFT']

{datetime.datetime(2020, 11, 20, 0, 0): ['10 Stocks Robinhood Users Are Trading During The Pandemic',
  'Markets On The Fence: Vaccine Optimism Vs. Worries About Virus, Stimulus, Fed Lending'],
 datetime.datetime(2020, 11, 19, 0, 0): ["Jim Cramer On Nvidia: So Much Demand They Can't Make Enough Chips",
  "Microsoft, Sony's Big Woe With PlayStation 5, New Xbox Series: High Demand, Very Low Supply"],
 datetime.datetime(2020, 11, 17, 0, 0): ['IPO News: Airbnb Finally Files To Go Public'],
 datetime.datetime(2020, 11, 16, 0, 0): ['This Big Fund Manager Is Betting On Nintendo In The Video Game Console Wars',
  'PlayStation 5 Might Outsell New Xbox Series But Gaming Subscription Service Gives Microsoft An Edge, Analysts Say'],
 datetime.datetime(2020, 11, 14, 0, 0): ['Google Invests In Indonesian E-Commerce Unicorn, Joins Singapore Government, SoftBank, Alibaba'],
 datetime.datetime(2020, 11, 13, 0, 0): ['TikTok Granted 15-Day Extension To Resolve National Security Concerns Before Sale',
  '

In [8]:
dill.dump(all_ticker_news, open('data/all_ticker_news.pkd', 'wb'))

### 2. Update StockTwits

In [7]:
cache_dir = 'data/STcache1'
!rm data/STcache1/*

In [8]:
@checkpoint(key=lambda args,kwargs:args[1], work_dir=cache_dir)
def get_st_messages(path,filename):
    response=get_response(path)
    x=response.json()
    messages=x['messages']
    d_x={}
    d_x['id'],d_x['message'],d_x['time'],d_x['username'],d_x['score'],d_x['sentiment']=[],[],[],[],[],[]
    for message in messages:
        d_x['id'].append(message['id'])
        d_x['message'].append(message['body'])
        d_x['time'].append(message['created_at'])
        d_x['username'].append(message['user']['username'])
        d_x['score'].append(message['user']['like_count'])
        if message['entities']['sentiment']!=None:
            d_x['sentiment'].append(message['entities']['sentiment']['basic'])
        else:
            d_x['sentiment'].append(None)
    return pd.DataFrame.from_dict(d_x, orient='columns')

In [9]:
d_ST_messages_alldat = dill.load(open('data/d_ST_messages_alldat.pkd', 'rb'))
d_ST_messages={}
for ticker in tickers:
    url='https://api.stocktwits.com/api/2/streams/symbol/'+ticker+'.json'
    try:
        d_ST_messages[ticker]=get_st_messages(url,ticker)
    except NameError:
        print('URL not found for', ticker)
d_ST_messages_alldat[datetime(datetime.now().year,datetime.now().month,datetime.now().day)]=d_ST_messages
dill.dump(d_ST_messages_alldat, open('data/d_ST_messages_alldat.pkd', 'wb'))

URL not found for FTV
URL not found for FBHS
URL not found for FOXA
URL not found for FOX
URL not found for BEN
URL not found for FCX
URL not found for GPS
URL not found for GRMN
URL not found for IT
URL not found for GD
URL not found for GE
URL not found for GIS
URL not found for GM
URL not found for GPC
URL not found for GILD
URL not found for GL
URL not found for GPN
URL not found for GS
URL not found for GWW
URL not found for HAL
URL not found for HBI
URL not found for HIG
URL not found for HAS
URL not found for HCA
URL not found for PEAK
URL not found for HSIC
URL not found for HSY
URL not found for HES
URL not found for HPE
URL not found for HLT
URL not found for HFC
URL not found for HOLX
URL not found for HD
URL not found for HON
URL not found for HRL
URL not found for HST
URL not found for HWM
URL not found for HPQ
URL not found for HUM
URL not found for HBAN
URL not found for HII
URL not found for IEX
URL not found for IDXX
URL not found for INFO
URL not found for ITW
URL not

In [10]:
d_ST_messages_alldat.keys()


dict_keys([datetime.datetime(2020, 10, 27, 0, 0), datetime.datetime(2020, 10, 31, 0, 0), datetime.datetime(2020, 10, 17, 0, 0), datetime.datetime(2020, 11, 6, 0, 0)])

### 3. Update daily price data

In [9]:
d_all_EOD = dill.load(open('data/d_all_EOD.pkd', 'rb'))
last_updated=d_all_EOD['SPY'].iloc[len(d_all_EOD['SPY'])-1]['date']
startdate=datetime.strftime(last_updated+timedelta(days=1),'%Y-%m-%d')
enddate=datetime.strftime(datetime.now(),'%Y-%m-%d')
startdate,enddate

('2020-11-10', '2020-11-21')

In [10]:
def get_EOD_data(symbol,sdate,edate):
    
    base_url='https://api.tiingo.com/tiingo/daily/'+symbol+'/prices?'
    api_key='9fa4df1e30e2fb224f1a625b7c43867b1880d05f'
    parameters={
        'token':api_key,
        'startDate':sdate,
        'endDate': edate,
    }
    response = requests.get(base_url, params=parameters)
    return response

In [11]:
errors=[]
for n,ticker in enumerate(tickers):
    try:
        df_EOD=pd.DataFrame(get_EOD_data(ticker,startdate,enddate).json())
        df_EOD['date']=df_EOD['date'].apply(lambda x:x[:10]).apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
        d_all_EOD[ticker]=d_all_EOD[ticker].append(df_EOD)
    except Exception as err:
        errors.append((ticker,err))

In [12]:
errors

[('BRK.B', ValueError('If using all scalar values, you must pass an index')),
 ('BF.B', ValueError('If using all scalar values, you must pass an index')),
 ('VNT', KeyError('VNT')),
 ('YUM', ValueError('If using all scalar values, you must pass an index')),
 ('ZBRA', ValueError('If using all scalar values, you must pass an index')),
 ('ZBH', ValueError('If using all scalar values, you must pass an index')),
 ('ZION', ValueError('If using all scalar values, you must pass an index')),
 ('ZTS', ValueError('If using all scalar values, you must pass an index'))]

In [13]:
d_all_EOD['SPY']

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2010-01-04,113.33,113.39,111.51,112.37,118857369,91.426173,91.474576,89.957933,90.651716,118857369,0.0,1.0
1,2010-01-05,113.63,113.68,112.85,113.26,111519230,91.668190,91.708526,91.038944,91.369702,111519230,0.0,1.0
2,2010-01-06,113.71,113.99,113.43,113.52,116017127,91.732728,91.958611,91.506845,91.579450,116017127,0.0,1.0
3,2010-01-07,114.19,114.33,113.18,113.50,130502017,92.119956,92.232898,91.305164,91.563316,130502017,0.0,1.0
4,2010-01-08,114.57,114.62,113.66,113.89,126357911,92.426512,92.466848,91.692392,91.877939,126357911,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2020-11-16,362.57,362.78,359.59,360.98,74541138,362.570000,362.780000,359.590000,360.980000,74541138,0.0,1.0
5,2020-11-17,360.62,361.92,358.34,359.97,66111009,360.620000,361.920000,358.340000,359.970000,66111009,0.0,1.0
6,2020-11-18,356.28,361.50,356.24,360.91,70591299,356.280000,361.500000,356.240000,360.910000,70591299,0.0,1.0
7,2020-11-19,357.78,358.18,354.15,355.60,59940947,357.780000,358.180000,354.150000,355.600000,59940947,0.0,1.0


In [14]:
dill.dump(d_all_EOD, open('data/d_all_EOD.pkd', 'wb'))
dill.dump(errors, open('data/errors.pkd', 'wb'))